Cargar archivos

In [17]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

observations_path = "observations_selection.csv"
shp_peninsula = r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos IEET\Mallas Península\Malla10km_p.shp"
shp_canarias = r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos IEET\Mallas Canarias\Malla10km_c.shp"

gdf_peninsula = gpd.read_file(shp_peninsula)
gdf_canarias = gpd.read_file(shp_canarias)

df = pd.read_csv(observations_path)


Pasar de coordenadas a geometría de puntos

In [18]:
# Crear geometría de puntos a partir de latitud y longitud
df["geometry"] = df.apply(lambda row: Point(row["decimalLongitude"], row["decimalLatitude"]), axis=1)

# Convertir DataFrame en GeoDataFrame
gdf_points = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # CRS WGS84


Transformar los sistemas de coordenadas

- Las coordenadas en el CSV están en WGS84 (EPSG:4326)
- Las cuadrículas SHP están en UTM (Husos 28-32, según la tabla).
- Necesitamos asegurarnos de que ambos usan el mismo sistema de coordenadas (CRS).

In [19]:
# Antes de cambiar sistemas de coordenadas
print("Points crs:" + str(gdf_points.crs))
print("Canarias crs:" + str(gdf_canarias.crs))
print("Peninsula crs:" + str(gdf_peninsula.crs))

# Usar CRS de EPSG:4326
gdf_points = gdf_points.to_crs(epsg=4326)
gdf_peninsula = gdf_peninsula.to_crs(epsg=4326)
gdf_canarias = gdf_canarias.to_crs(epsg=4326)

# Después
print("Points crs:" + str(gdf_points.crs))
print("Canarias crs:" + str(gdf_canarias.crs))
print("Peninsula crs:" + str(gdf_peninsula.crs))


Points crs:EPSG:4326
Canarias crs:EPSG:32628
Peninsula crs:EPSG:25830
Points crs:EPSG:4326
Canarias crs:EPSG:4326
Peninsula crs:EPSG:4326


Realizar unión espacial para asignar cada punto a una cuadrícula de la malla

In [20]:
# Unión espacial (spatial join) de puntos con cuadrículas de la Península.
# how="left" hace que se incluyan todos los puntos y solo agrega información de la cuadrícula si hay intersección
# predicate="within" solo asigna un punto a una cuadrícula si está dentro de ella.
gdf_result_peninsula = gpd.sjoin(gdf_points, gdf_peninsula, how="left", predicate="within")
gdf_result_canarias = gpd.sjoin(gdf_points, gdf_canarias, how="left", predicate="within")

# Unir los resultados (usando combine_first para dar preferencia a la Península)
gdf_result = gdf_result_peninsula.combine_first(gdf_result_canarias)


# Unir resultados (primero Península, luego Canarias para puntos no asignados)
# Toma valores de gdf_result_peninsula primero y, si hay valores NaN (es decir, si un punto no fue asignado a la Península), usa los de gdf_result_canarias.
gdf_result = gdf_result_peninsula.combine_first(gdf_result_canarias)


Guardar resultados

In [21]:
df['malla_codigo'] = gdf_result['CUADRICULA']

# Guardar el CSV con la nueva columna de códigos de malla en 2

midpoint = len(df) // 2

df_part1 = df.iloc[:midpoint]
df_part2 = df.iloc[midpoint:]

# Save them as separate CSV files
df_part1.to_csv("observations_con_cuadricula_1.csv", index=False)
df_part2.to_csv("observations_con_cuadricula_2.csv", index=False)